Уральсков Евгений ПИ18-2

Датасет: https://www.kaggle.com/austinreese/craigslist-carstrucks-data  
Данные о ценах на автомобили на вторичном рынке

Импорт библиотек

In [115]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings("ignore")

Загрузка и просмотр структуры данных

In [116]:
df = pd.read_csv('vehicles.csv')
print(f'Кол-во записей: {len(df)}')
print(f'Кол-во колонок: {len(df.columns)}')
df.head()

Кол-во записей: 458213
Кол-во колонок: 26


,Unnamed: 0,id,url,region,region_url,price,year,manufacturer,model,condition,...,drive,size,type,paint_color,image_url,description,state,lat,long,posting_date
0,0,7240372487,https://auburn.craigslist.org/ctd/d/auburn-uni...,auburn,https://auburn.craigslist.org,35990,2010.0,chevrolet,corvette grand sport,good,...,rwd,NaN,other,NaN,https://images.craigslist.org/00N0N_ipkbHVZYf4...,Carvana is the safer way to buy a car During t...,al,32.590000,-85.480000,2020-12-02T08:11:30-0600
1,1,7240309422,https://auburn.craigslist.org/cto/d/auburn-201...,auburn,https://auburn.craigslist.org,7500,2014.0,hyundai,sonata,excellent,...,fwd,NaN,sedan,NaN,https://images.craigslist.org/00s0s_gBHYmJ5o7y...,I'll move to another city and try to sell my c...,al,32.547500,-85.468200,2020-12-02T02:11:50-0600
2,2,7240224296,https://auburn.craigslist.org/cto/d/auburn-200...,auburn,https://auburn.craigslist.org,4900,2006.0,bmw,x3 3.0i,good,...,NaN,NaN,SUV,blue,https://images.craigslist.org/00B0B_5zgEGWPOrt...,Clean 2006 BMW X3 3.0I. Beautiful and rare Bl...,al,32.616807,-85.464149,2020-12-01T19:50:41-0600
3,3,7240103965,https://auburn.craigslist.org/cto/d/lanett-tru...,auburn,https://auburn.craigslist.org,2000,1974.0,chevrolet,c-10,good,...,rwd,full-size,pickup,blue,https://images.craigslist.org/00M0M_6o7KcDpArw...,1974 chev. truck (LONG BED) NEW starter front ...,al,32.861600,-85.216100,2020-12-01T15:54:45-0600
4,4,7239983776,https://auburn.craigslist.org/cto/d/auburn-200...,auburn,https://auburn.craigslist.org,19500,2005.0,ford,f350 lariat,excellent,...,4wd,full-size,pickup,blue,https://images.craigslist.org/00p0p_b95l1EgUfl...,2005 Ford F350 Lariat (Bullet Proofed). This t...,al,32.547500,-85.468200,2020-12-01T12:53:56-0600


Наименования колонок

In [117]:
df.columns

Index(['Unnamed: 0', 'id', 'url', 'region', 'region_url', 'price', 'year',
       'manufacturer', 'model', 'condition', 'cylinders', 'fuel', 'odometer',
       'title_status', 'transmission', 'VIN', 'drive', 'size', 'type',
       'paint_color', 'image_url', 'description', 'state', 'lat', 'long',
       'posting_date'],
      dtype='object')

**id** - id объявления  
**url** - адрес объявления  
**region** - название региона продажи  
**region_url** - адрес страницы с объявлениями о продаже для данного региона  
**price** - цена  
**year** - год выпуска автомобиля  
**manufacturer** - марка автомобиля  
**model** - модель автомобиля  
**condition** - состояние автомобиля  
**cylinders** - количество цилиндров  
**fuel** - тип топлива  
**odometer** - пробег  
**title_status** - "статус" автомобиля (чистая, грязная, на запчасти и т.д.)  
**transmission** - тип трансмиссии (механика, автомат)  
**vin** - vin-номер, уникальный номер автомобиля  
**drive** - привод  
**size** - размер автомобиля  
**type** - тип кузова  
**paint_color** - цвет автомобиля  
**image_url** - ссылка на изображение  
**description** - описание  
**county** - страна продажи (всегда США)  
**state** - штат  
**lat, long** - координаты объявления  

Удаляем то что точно не понадобится

In [118]:
df = df[['id', 'region', 'price', 'year', 'manufacturer', 'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status',
        'transmission', 'drive', 'size', 'type', 'paint_color', 'description', 'state']]
df.head()

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,description,state
0,7240372487,auburn,35990,2010.0,chevrolet,corvette grand sport,good,8 cylinders,gas,32742.0,clean,other,rwd,NaN,other,NaN,Carvana is the safer way to buy a car During t...,al
1,7240309422,auburn,7500,2014.0,hyundai,sonata,excellent,4 cylinders,gas,93600.0,clean,automatic,fwd,NaN,sedan,NaN,I'll move to another city and try to sell my c...,al
2,7240224296,auburn,4900,2006.0,bmw,x3 3.0i,good,6 cylinders,gas,87046.0,clean,automatic,NaN,NaN,SUV,blue,Clean 2006 BMW X3 3.0I. Beautiful and rare Bl...,al
3,7240103965,auburn,2000,1974.0,chevrolet,c-10,good,4 cylinders,gas,190000.0,clean,automatic,rwd,full-size,pickup,blue,1974 chev. truck (LONG BED) NEW starter front ...,al
4,7239983776,auburn,19500,2005.0,ford,f350 lariat,excellent,8 cylinders,diesel,116000.0,lien,automatic,4wd,full-size,pickup,blue,2005 Ford F350 Lariat (Bullet Proofed). This t...,al


Смотрим количество пропущенных значений по столбцам

In [119]:
df.isna().sum()

id                   0
region               0
price                0
year              1050
manufacturer     18220
model             4846
condition       192940
cylinders       171140
fuel              3237
odometer         55303
title_status      2577
transmission      2442
drive           134188
size            321348
type            112738
paint_color     140843
description         70
state                0
dtype: int64

Марка автомобиля - один из важнейших параметров, однако, некоторые люди могли по ошибке указать марку в графе модели или описании, поэтому сперва удалим записи, где отсутствуют все 3 значения

In [120]:
df_len = len(df)
df = df[df['manufacturer'].notna() | df['description'].notna() | df['model'].notna()]
print(f'Удалено записей: {df_len-len(df)}')

Удалено записей: 28


Также удалим записи, где не указаны пробег, год выпуска и привод. Также удалим записи с нулевой ценой.

In [121]:
df_len = len(df)
df = df[df['price']!=0]
df = df[df['year'].notna() & df['odometer'].notna() & df['transmission'].notna()]
print(f'Удалено записей: {df_len-len(df)}')

Удалено записей: 85404


Смотрим количество пропущенных значений по столбцам

In [122]:
df.isna().sum()

id                   0
region               0
price                0
year                 0
manufacturer     11511
model             2668
condition       139492
cylinders       121451
fuel              2231
odometer             0
title_status      1399
transmission         0
drive            92236
size            256426
type             71824
paint_color      95108
description         42
state                0
dtype: int64

Посмотрим, есть ли у нас названия марок в полях моделей.

In [123]:
df['model'] = df['model'].str.lower()
df['manufacturer'] = df['manufacturer'].str.lower()
df['description'] = df['description'].str.lower()

manufacturers = df['manufacturer'].unique()
manufacturers = [i for i in manufacturers if i is not np.nan]
for i in manufacturers:
    print(i)
    print(df['model'].str.contains(i).value_counts())
    print()

chevrolet
False    370097
True         16
Name: model, dtype: int64

hyundai
False    370109
True          4
Name: model, dtype: int64

bmw
False    370072
True         41
Name: model, dtype: int64

ford
False    370042
True         71
Name: model, dtype: int64

toyota
False    370092
True         21
Name: model, dtype: int64

jeep
False    370080
True         33
Name: model, dtype: int64

ram
False    368718
True       1395
Name: model, dtype: int64

honda
False    370060
True         53
Name: model, dtype: int64

dodge
False    370106
True          7
Name: model, dtype: int64

lexus
False    370096
True         17
Name: model, dtype: int64

mercedes-benz
False    370103
True         10
Name: model, dtype: int64

cadillac
False    370107
True          6
Name: model, dtype: int64

gmc
False    370066
True         47
Name: model, dtype: int64

subaru
False    370109
True          4
Name: model, dtype: int64

mazda
False    368714
True       1399
Name: model, dtype: int64

mini
False    

Да, такие ошибки присутствуют, но у марок "ram" и "mini" таких совпадений подозрительно много. Скорее всего данные буквосочетания реально могут встречаться в названии модели, поэтому не будем трогать данные марки, а в остальных случаях извлечем названия марки.

In [ ]:
manufacturers = [i for i in manufacturers if (i != 'ram') and (i != 'mini')]

for i in manufacturers:
    contain_man = df['model'].str.contains(i)
    contain_man[contain_man.isna()] = False
    df.loc[contain_man, 'manufacturer'] = i
    df.loc[contain_man, 'model'] = df[contain_man]['model'].str.replace(i, '', regex = False)

Удалим пробелы, звездочки и прочую ерунду из моделей автомобилей. На мой взгляд, нам не столь важны точные названия, но зато это сильно их унифицирует. То что из-за этого у нас совпадут разные модели также маловероятно.

In [ ]:
df['model'] = df['model'].str.replace('\W', '')

Теперь попробуем вычленить марки из описания, однако организуем проверку на то, чтобы описание содержало только одну марку автомобиля, потому что на данном сайте присутствуют автосалоны, которые указывают в описании всевозможные марки автомобилей.

In [ ]:
man_in_description = df[df['manufacturer'].isna()][['id', 'description']].assign(manufacturers=0)
for i in manufacturers:
    contain_man = man_in_description['description'].str.contains(i)
    contain_man[contain_man.isna()] = False
    man_in_description.loc[contain_man, 'manufacturers'] += 1
    
man_in_description = man_in_description[man_in_description['manufacturers']==1]['id']
print('Количество записей, с одной маркой в описании: ', len(man_in_description))

for i in manufacturers:
    contain_man = df['description'].str.contains(i)
    contain_man[contain_man.isna()] = False
    df.loc[contain_man & df['id'].isin(man_in_description.tolist()), 'manufacturer'] = i
    df.loc[contain_man, 'description'] = df[contain_man]['description'].str.replace(i, '', regex = False)

Количество записей, с одной маркой в описании:  2146


In [ ]:
df['manufacturer'].isna().sum()

Сделал все что мог, удаляю записи с незаполненной маркой, и так же 2 записи без типа топлива. Удаляю колонку "description", больше она не понадобится.

In [ ]:
df = df[df['manufacturer'].notna()]
df = df[df['fuel'].notna()]
df = df.drop(columns=['description'])
df.isna().sum()

Разберемся с цветом автомобиля.

In [ ]:
df['paint_color'].value_counts(dropna=False)

Заполним отсутствующие значения как "unknown"

In [ ]:
df.loc[df['paint_color'].isna(), 'paint_color']='unknown'

Сделаем то же самое для колонки "condition"

In [ ]:
df.loc[df['condition'].isna(), 'condition']='unknown'

In [ ]:
df.isna().sum()

In [ ]:
df['type'].unique()

In [ ]:
df['drive'].unique()

In [ ]:
df['size'].unique()

По-идее, тип привода и кузова не должны отличаться для одной модели автомобиля, поэтому заполним отсутствующие значения данными из других записей, содержащимися наибольшее количество раз для одной модели. Возьмем модели, которые встречаются хотя бы 30 раз.

Автомобили,конечно, могут быть с полным/задним/передним приводом в зависимости от комплектации.

In [ ]:
popular_models = df.assign(mod=df['model']+' '+df['manufacturer'])['mod'].value_counts().reset_index()
popular_models.columns = ['model', 'count']
popular_models = popular_models[popular_models['count']>=30]
popular_models.tail()

In [ ]:
for i in popular_models['model'].tolist():
    model = i.split()
    drive = df[(df['model']==model[0]) & (df['manufacturer']==model[1])]['drive'].value_counts(normalize = True).reset_index()
    drive = drive['index']
    if len(drive)>0:
        df.loc[(df['model']==model[0]) & (df['manufacturer']==model[1]) & (df['drive'].isna()), 'drive'] = drive[0]

In [ ]:
for i in popular_models['model'].tolist():
    model = i.split()
    car_type = df[(df['model']==model[0]) & (df['manufacturer']==model[1])]['type'].value_counts(normalize = True).reset_index()
    car_type = car_type['index']
    if len(car_type)>0:
        df.loc[(df['model']==model[0]) & (df['manufacturer']==model[1]) & (df['type'].isna()), 'type'] = car_type[0]

In [ ]:
for i in popular_models['model'].tolist():
    model = i.split()
    size = df[(df['model']==model[0]) & (df['manufacturer']==model[1])]['size'].value_counts(normalize = True).reset_index()
    size = size['index']
    if len(size)>0:
        df.loc[(df['model']==model[0]) & (df['manufacturer']==model[1]) & (df['size'].isna()), 'size'] = size[0]

In [ ]:
df.isna().sum()

In [ ]:
df = df[df['drive'].notna()]
df = df[df['type'].notna()]
df = df[df['size'].notna()]
df = df[df['title_status'].notna()]
df.isna().sum()

In [ ]:
len(df)

In [ ]:
print(f'Количество уникальных моделей автомобилей: {len(df["model"].unique())}')

Работать с таким количеством моделей не представляется возможным, да и в теории, модель автомобиля должна определяться его маркой и характеристиками, поэтому удалим эту колонку. Также давно пора было удалить id, т.к. для нас оно не несет смысловой нагрузки.

In [ ]:
df=df.drop(columns=['id', 'model'])

In [ ]:
df.isna().sum()

Приведем год и пробег к целочисленному типу.

In [ ]:
df['year']=df['year'].astype(int)
df['odometer']=df['odometer'].astype(int)

In [ ]:
df['manufacturer'].value_counts()

In [ ]:
unpopular_manufacturers = df['manufacturer'].value_counts().reset_index()
unpopular_manufacturers = unpopular_manufacturers[unpopular_manufacturers['manufacturer']<=300]
unpopular_manufacturers = unpopular_manufacturers['index'].tolist()
df.loc[df['manufacturer'].isin(unpopular_manufacturers), 'manufacturer'] = 'other'

df['manufacturer'].value_counts()

In [ ]:
df.loc[df.region.isin(df['region'].value_counts().nlargest(35).index), 'region'] = 4
df.loc[df.region.isin(df[df['region']!=4]['region'].value_counts().nlargest(36).index), 'region'] = 3
df.loc[df.region.isin(df[~df['region'].isin([3, 4])]['region'].value_counts().nlargest(59).index), 'region'] = 2
df.loc[~df['region'].isin([2, 3, 4]), 'region'] = 1

df[df['cylinders'] == '5 cylinders'] = 'other'
df[df['cylinders'] == '10 cylinders'] = 'other'
df[df['cylinders'] == '3 cylinders'] = 'other'
df[df['cylinders'] == '12 cylinders'] = 'other'

for col in ['region', 'manufacturer', 'condition', 'cylinders', 
            'fuel', 'title_status', 'transmission', 'drive', 'size', 'type', 'paint_color', 'stat
    df[col] = df[col].astype('category')

In [ ]:
df.dtypes

График цены

In [ ]:
sns_plot = sns.distplot(df['price'])
fig = sns_plot.get_figure()

In [ ]:
df['price'].describe()

Какой-то ужас, максимальная цена 2.5 миллиарда

In [ ]:
sns_plot = sns.distplot(df['price'][df['price']<70000])
fig = sns_plot.get_figure()

Удаляем выбросы

In [ ]:
df = df[df['price']<70000]

In [ ]:
for i in ['manufacturer', 'condition', 'cylinders', 'fuel', 'title_status',
        'transmission', 'drive', 'type', 'paint_color', 'state']:
    df[df['price']<70000].groupby(i)["price"].mean().sort_values(ascending=False).plot.bar(figsize=(16,5));
    plt.title('Средняя цена в зависимости от ' + i)
    plt.show();

In [ ]:
sns.heatmap(df.corr())

In [ ]:
df[df['transmission']=='other']

In [ ]:
df[df['odometer']<2000000].plot.scatter(x='odometer', y='price', figsize=(9,6),)
plt.title('\nЗависимость цены от пробега\n', fontsize=14, fontweight='bold');

In [ ]:
df.plot.scatter(x='year', y='price', figsize=(9,6),)
plt.title('\nЗависимость цены от года выпуска\n', fontsize=14, fontweight='bold');

In [ ]:
df.head()

In [ ]:
df.isna().sum()

# ПРОБУЮ ПРЕДСКАЗАТЬ ЦИЛИНДРЫ, ЗАПУСКАТЬ НА СВОЙ СТРАХ И РИСК

In [ ]:
target = df[['price']]
features = df.drop(['cylinders', 'price'], axis=1)

In [ ]:
num_feats = features.select_dtypes(include=['int32', 'int64']).copy()
cat_feats = features.select_dtypes(include=['category']).copy()
cat_feats = pd.get_dummies(cat_feats)

In [ ]:
features_recoded = pd.concat([num_feats, df['cylinders'], cat_feats], axis=1)
features_recoded.head()

In [ ]:
df['cylinders'].value_counts()

In [ ]:
df.to_csv('vehicles_clear.csv')
# df_new = pd.read_csv('vehicles_clear.csv')
# df_new=df_new.drop(columns=['Unnamed: 0'])

In [ ]:
train_data = features_recoded[features_recoded['cylinders'].notnull() & ~features_recoded['cylinders'].isin(['other'])]
test_data  = features_recoded[features_recoded['cylinders'].isnull() & features_recoded['cylinders'].isin(['other'])]

X_train, X_test, y_train, y_test = train_test_split(
    train_data.drop('cylinders', axis=1), train_data['cylinders'], test_size=0.33, random_state=42)

In [ ]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
print(regr.score(X_train, y_train))
print(regr.score(X_test, y_test))
predicted = regr.predict(X_test)
print(accuracy_score(y_test, predicted))

In [ ]:
print(predicted)

Достаточно неплохо получилось предсказать, теперь попробую грид, чтобы немного улучшить результат.

In [ ]:
clf = RandomForestClassifier()
param_grid = { 
    'n_estimators': [200, 300, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [5,6,7],
}
CV_rfc = GridSearchCV(estimator=regr, param_grid=param_grid, cv= 2, n_jobs=-1, verbose=True)
CV_rfc.fit(X_train, y_train)
CV_rfc.best_params_

In [ ]:
print(CV_rfc.score(X_train, y_train))
print(CV_rfc.score(X_test, y_test))
predicted = CV_rfc.predict(X_test)
predicted = pd.DataFrame(predicted)
predicted.columns = ['cylinders']
print(predicted.head())
print(accuracy_score(y_test, predicted))

In [ ]:
clf = RandomForestClassifier(n_estimators= , max_features= , max_depth= )
clf.fit(train_data.drop('cylinders', axis=1), train_data['cylinders'])
predicted = clf.predict(test_data)
print(predicted)

In [ ]:
sub_df_new = pd.concat([new_test_data, train_data], axis=0)

print(sub_df_new.shape)
sub_df_new.head()

In [ ]:
prelim_index = pd.DataFrame(test_data.drop('size', axis=1).index)
prelim_index.columns = ['prelim']

predicted = pd.concat([predicted, prelim_index], axis=1)
predicted.set_index(['prelim'], inplace=True)
predicted.head()
